### Tokenize reviews ###

### Import Dependencies ###

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, MatchAny, FieldCondition, Filter, Prefetch, FusionQuery

import pandas as pd
import numpy as np
import openai
import json
import tiktoken

/Users/adityanatani/Documents/workspace/ai-engineering-bootcamp-prerequisite/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
dummy_vector = np.zeros(1536).tolist()

In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [5]:
payload.points

[ScoredPoint(id=273, version=3, score=0.0, payload={'parent_asin': 'B0C7CPL179'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=345, version=3, score=0.0, payload={'parent_asin': 'B09W5RJ1LY'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=399, version=3, score=0.0, payload={'parent_asin': 'B09KZK1494'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2, version=3, score=0.0, payload={'parent_asin': 'B09JV5FM2S'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=469, version=3, score=0.0, payload={'parent_asin': 'B0BTKWY6XG'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=225, version=3, score=0.0, payload={'parent_asin': 'B098K6N6TX'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=19, version=3, score=0.0, payload={'parent_asin': 'B09XX3J786'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=360, version=3, score=0.0, payload={'parent_asin': 'B09QZSHCKS'}, vector=None

In [6]:
len(payload.points)

1000

In [8]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]

In [9]:
parent_asin_list

['B0C7CPL179',
 'B09W5RJ1LY',
 'B09KZK1494',
 'B09JV5FM2S',
 'B0BTKWY6XG',
 'B098K6N6TX',
 'B09XX3J786',
 'B09QZSHCKS',
 'B0BG54HV9X',
 'B0B39PFLSW',
 'B09XXW175L',
 'B0BXRSFB93',
 'B0BY2BBDS9',
 'B0C9XFF3CT',
 'B0B6894XGM',
 'B0B7ML1ZGV',
 'B0BB2KFPBJ',
 'B0BTDJ6FBR',
 'B09QGNB537',
 'B0B6J4Y4QX',
 'B0B1F4FKPF',
 'B0BYSRTQNQ',
 'B0BT6SCPW7',
 'B0B965NYT9',
 'B0B7NMSTT9',
 'B09MLGP9SP',
 'B09TX32DMD',
 'B0C7492QYK',
 'B0C61FTDQR',
 'B08M65W3PT',
 'B0C3VRH58B',
 'B0C94HPBL2',
 'B0BJJYFRLF',
 'B0BGGH27H9',
 'B0B678PFZ6',
 'B0BXPJWMNY',
 'B0B5517LRN',
 'B09YJ2867L',
 'B09QPQZMMN',
 'B09CTF6PB6',
 'B0C78HVVL2',
 'B09YB3ZMRT',
 'B0BYN7PTKX',
 'B09RV6KSMH',
 'B0BYYLJRHT',
 'B09MPNX216',
 'B0B4B45CYV',
 'B09QG1CC6H',
 'B0C9TBMMFN',
 'B0B2V9P3K4',
 'B0BWTVXQ62',
 'B0C2C87FW3',
 'B015TPLFFO',
 'B08V8R6JML',
 'B0BYKDMMVJ',
 'B0C4DYTQH8',
 'B0CF1WM24K',
 'B0B9JF64DM',
 'B0CB5N7PND',
 'B0BM9MTSH2',
 'B0BJV3VJC6',
 'B0CC61QC4J',
 'B0B4PW3CR9',
 'B09NLTDHQ6',
 'B0BHT7PYNM',
 'B0C16S18PX',
 'B09XXL21

In [10]:
len(parent_asin_list)

1000

### Load Amazon Review Dataset ###

In [27]:
df_reviews = pd.read_json("../../data/Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)


In [28]:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Perfect!,This is perfect! Thank you so much!!! I absolu...,[],B09992M2LX,B09ZPV8WBV,AHX4XWVVQUKT3FCNWCVASDF4Q56Q,2022-08-05 04:06:39.589,0,True
1,5,3ft mini usb cables,I don't have many things that still use a mini...,[],B09Y94B2NM,B09Y95BMKX,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-16 16:03:28.714,3,True
2,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True
3,5,Great to Have Around,My husband and I were recently working a booth...,[],B09M89JN7B,B0BYYGZHG5,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2022-03-22 01:43:49.342,0,False
4,5,Easy to use,Work as advertised and at a very good price.,[],B07T55DL33,B0B2JWCMCY,AG7WKTZINOFIXMZJYIPKIB7PV7NQ,2019-12-28 06:12:24.960,0,True


In [29]:
len(df_reviews)

105918

In [30]:
df_review_sample = df_reviews[df_reviews["parent_asin"].isin(parent_asin_list)]

In [31]:
df_review_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Perfect!,This is perfect! Thank you so much!!! I absolu...,[],B09992M2LX,B09ZPV8WBV,AHX4XWVVQUKT3FCNWCVASDF4Q56Q,2022-08-05 04:06:39.589,0,True
1,5,3ft mini usb cables,I don't have many things that still use a mini...,[],B09Y94B2NM,B09Y95BMKX,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-16 16:03:28.714,3,True
2,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True
3,5,Great to Have Around,My husband and I were recently working a booth...,[],B09M89JN7B,B0BYYGZHG5,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2022-03-22 01:43:49.342,0,False
4,5,Easy to use,Work as advertised and at a very good price.,[],B07T55DL33,B0B2JWCMCY,AG7WKTZINOFIXMZJYIPKIB7PV7NQ,2019-12-28 06:12:24.960,0,True


In [32]:
len(df_review_sample)

105918

### Define functions to preprocess reviews data ###

In [33]:
def preprocess_review_data(row):
    return f"{row['title']} {row['text']}"

In [34]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

In [35]:
encoding.encode("Can I get some earphones")

[6854, 358, 636, 1063, 2487, 17144]

In [36]:
def token_count(row, model="text-embedding-3-small"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(row["preprocessed_data"]))

In [37]:
df_review_sample["preprocessed_data"] = df_review_sample.apply(preprocess_review_data,axis=1)

In [42]:
df_review_sample["preprocessed_data_token_count"] = df_review_sample.apply(token_count, axis=1)

In [43]:
df_review_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocessed_data,preprocessed_data_token_count
0,5,Perfect!,This is perfect! Thank you so much!!! I absolu...,[],B09992M2LX,B09ZPV8WBV,AHX4XWVVQUKT3FCNWCVASDF4Q56Q,2022-08-05 04:06:39.589,0,True,Perfect! This is perfect! Thank you so much!!!...,21
1,5,3ft mini usb cables,I don't have many things that still use a mini...,[],B09Y94B2NM,B09Y95BMKX,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-16 16:03:28.714,3,True,3ft mini usb cables I don't have many things t...,114
2,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True,I would buy it again. Great product. Worked we...,24
3,5,Great to Have Around,My husband and I were recently working a booth...,[],B09M89JN7B,B0BYYGZHG5,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2022-03-22 01:43:49.342,0,False,Great to Have Around My husband and I were rec...,76
4,5,Easy to use,Work as advertised and at a very good price.,[],B07T55DL33,B0B2JWCMCY,AG7WKTZINOFIXMZJYIPKIB7PV7NQ,2019-12-28 06:12:24.960,0,True,Easy to use Work as advertised and at a very g...,13


In [39]:
len(df_review_sample)

105918

In [45]:
df_review_sample = df_review_sample[df_review_sample["preprocessed_data_token_count"] < 8192]

In [46]:
len(df_review_sample)

105918

In [47]:
total_tokens = df_review_sample["preprocessed_data_token_count"].sum()

In [48]:
total_tokens

np.int64(6204437)

### Create New Qudrant Client Collection for Reviews ###

In [49]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-reviews",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [50]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-reviews",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

### Embedding Functions ###

In [51]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [52]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):
    
    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter * batch_size} of {len(text_list)}")
        counter += 1
    
    return all_embeddings

### Embed the text and add additional fields to the payload of each vector for reviews ###

In [53]:
data_to_embed_reviews = df_review_sample[["preprocessed_data", "parent_asin"]].to_dict(orient="records")

In [54]:
data_to_embed_reviews

[{'preprocessed_data': 'Perfect! This is perfect! Thank you so much!!! I absolutely love it!!! It’s great quality!!!',
  'parent_asin': 'B09ZPV8WBV'},
 {'preprocessed_data': "3ft mini usb cables I don't have many things that still use a mini USB charger cable, but I after a while you have to replace the charger cables.  These seem to work well and it was noted in the ad that they are reinforced at the head area to prolong life.  We shall see - time will tell.  I never hesitate to update my reviews should new info seem useful.  I do not accept any discounts or deals that are not available to all shoppers. And my reviews are based purely on my personal experience with each item I review.",
  'parent_asin': 'B09Y95BMKX'},
 {'preprocessed_data': 'I would buy it again. Great product. Worked well for what we needed it for. Would buy it again.',
  'parent_asin': 'B0B2JWCMCY'},
 {'preprocessed_data': 'Great to Have Around My husband and I were recently working a booth at a trade show for his b

In [55]:
text_to_embed_reviews = [data["preprocessed_data"] for data in data_to_embed_reviews]

In [56]:
text_to_embed_reviews

['Perfect! This is perfect! Thank you so much!!! I absolutely love it!!! It’s great quality!!!',
 "3ft mini usb cables I don't have many things that still use a mini USB charger cable, but I after a while you have to replace the charger cables.  These seem to work well and it was noted in the ad that they are reinforced at the head area to prolong life.  We shall see - time will tell.  I never hesitate to update my reviews should new info seem useful.  I do not accept any discounts or deals that are not available to all shoppers. And my reviews are based purely on my personal experience with each item I review.",
 'I would buy it again. Great product. Worked well for what we needed it for. Would buy it again.',
 'Great to Have Around My husband and I were recently working a booth at a trade show for his business. I set up a charging station with these. It was perfect! We were able to accommodate everyone who needed to charge their phone! The cables are a good length, not too short or t

In [57]:
embeddings_reviews = get_embeddings_batch(text_to_embed_reviews, batch_size=500)

Processed 500 of 105918
Processed 1000 of 105918
Processed 1500 of 105918
Processed 2000 of 105918
Processed 2500 of 105918
Processed 3000 of 105918
Processed 3500 of 105918
Processed 4000 of 105918
Processed 4500 of 105918
Processed 5000 of 105918
Processed 5500 of 105918
Processed 6000 of 105918
Processed 6500 of 105918
Processed 7000 of 105918
Processed 7500 of 105918
Processed 8000 of 105918
Processed 8500 of 105918
Processed 9000 of 105918
Processed 9500 of 105918
Processed 10000 of 105918
Processed 10500 of 105918
Processed 11000 of 105918
Processed 11500 of 105918
Processed 12000 of 105918
Processed 12500 of 105918
Processed 13000 of 105918
Processed 13500 of 105918
Processed 14000 of 105918
Processed 14500 of 105918
Processed 15000 of 105918
Processed 15500 of 105918
Processed 16000 of 105918
Processed 16500 of 105918
Processed 17000 of 105918
Processed 17500 of 105918
Processed 18000 of 105918
Processed 18500 of 105918
Processed 19000 of 105918
Processed 19500 of 105918
Proces

In [58]:
len(embeddings_reviews)

105918

In [59]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings_reviews, data_to_embed_reviews):
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocessed_data"],
                "parent_asin": data["parent_asin"],
            }
        )
    )
    i += 1

In [60]:
batch_size_qdrant = 100
counter = 1
for i in range(0, len(pointstructs), batch_size_qdrant):
    batch = pointstructs[i:i + batch_size_qdrant]
    qdrant_client.upsert(
        collection_name="Amazon-items-collection-01-reviews",
        wait=True,
        points=batch
    )
    print(f"Processed {counter * batch_size_qdrant} of {len(pointstructs)}")
    counter += 1

Processed 100 of 105918
Processed 200 of 105918
Processed 300 of 105918
Processed 400 of 105918
Processed 500 of 105918
Processed 600 of 105918
Processed 700 of 105918
Processed 800 of 105918
Processed 900 of 105918
Processed 1000 of 105918
Processed 1100 of 105918
Processed 1200 of 105918
Processed 1300 of 105918
Processed 1400 of 105918
Processed 1500 of 105918
Processed 1600 of 105918
Processed 1700 of 105918
Processed 1800 of 105918
Processed 1900 of 105918
Processed 2000 of 105918
Processed 2100 of 105918
Processed 2200 of 105918
Processed 2300 of 105918
Processed 2400 of 105918
Processed 2500 of 105918
Processed 2600 of 105918
Processed 2700 of 105918
Processed 2800 of 105918
Processed 2900 of 105918
Processed 3000 of 105918
Processed 3100 of 105918
Processed 3200 of 105918
Processed 3300 of 105918
Processed 3400 of 105918
Processed 3500 of 105918
Processed 3600 of 105918
Processed 3700 of 105918
Processed 3800 of 105918
Processed 3900 of 105918
Processed 4000 of 105918
Processed

In [61]:
def retrieve_prefiltered_reviews_data(query, parent_asins, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-reviews",
        prefetch=[
            Prefetch(
                query=query_embedding,
                filter=Filter(
                    must=[
                        FieldCondition(
                            key="parent_asin",
                            match=MatchAny(
                                any=parent_asins
                            )
                        )
                    ]
                ),
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    return results

In [62]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G"], k=5)

In [63]:
reviews.points

[ScoredPoint(id=82510, version=828, score=0.5, payload={'text': 'Fans with great pontential, but overall a half baked product. I ordered two of these fans and both were defective.<br /><br />-Fan 1 cannot stick to the PMW profile and speeds constantly surge up and down a couple of hundred RPM every few seconds. Makes annoying sounds as it constantly surges.<br /><br />-Fan two seems to have some factory fault with the bearing where the center hub sort of wobbles and makes an annoying sound like something is out of tolerance every few seconds at all RPM ranges.', 'parent_asin': 'B09VDNKL4G'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=95165, version=954, score=0.33333334, payload={'text': 'Maybe a bad initial batch? They are very annoyingly noisey. I was so excited to get these after watching the youtube reviews at launch. Turns out I am in the same boat as other Amazon reviewers who are experiencing a lot of periodic resonance noise that is extremely annoying and w

In [64]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G", "B09YD1FZDN"], k=5)

In [65]:
reviews.points

[ScoredPoint(id=82510, version=828, score=0.5, payload={'text': 'Fans with great pontential, but overall a half baked product. I ordered two of these fans and both were defective.<br /><br />-Fan 1 cannot stick to the PMW profile and speeds constantly surge up and down a couple of hundred RPM every few seconds. Makes annoying sounds as it constantly surges.<br /><br />-Fan two seems to have some factory fault with the bearing where the center hub sort of wobbles and makes an annoying sound like something is out of tolerance every few seconds at all RPM ranges.', 'parent_asin': 'B09VDNKL4G'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=95165, version=954, score=0.33333334, payload={'text': 'Maybe a bad initial batch? They are very annoyingly noisey. I was so excited to get these after watching the youtube reviews at launch. Turns out I am in the same boat as other Amazon reviewers who are experiencing a lot of periodic resonance noise that is extremely annoying and w

In [66]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G", "B09YD1FZDN"], k=20)

In [67]:
reviews.points

[ScoredPoint(id=82510, version=828, score=0.5, payload={'text': 'Fans with great pontential, but overall a half baked product. I ordered two of these fans and both were defective.<br /><br />-Fan 1 cannot stick to the PMW profile and speeds constantly surge up and down a couple of hundred RPM every few seconds. Makes annoying sounds as it constantly surges.<br /><br />-Fan two seems to have some factory fault with the bearing where the center hub sort of wobbles and makes an annoying sound like something is out of tolerance every few seconds at all RPM ranges.', 'parent_asin': 'B09VDNKL4G'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=95165, version=954, score=0.33333334, payload={'text': 'Maybe a bad initial batch? They are very annoyingly noisey. I was so excited to get these after watching the youtube reviews at launch. Turns out I am in the same boat as other Amazon reviewers who are experiencing a lot of periodic resonance noise that is extremely annoying and w

In [70]:
for point in reviews.points:
    print(point.payload["parent_asin"])
    print(point.payload["text"])
    print("-"*100)


B09VDNKL4G
Fans with great pontential, but overall a half baked product. I ordered two of these fans and both were defective.<br /><br />-Fan 1 cannot stick to the PMW profile and speeds constantly surge up and down a couple of hundred RPM every few seconds. Makes annoying sounds as it constantly surges.<br /><br />-Fan two seems to have some factory fault with the bearing where the center hub sort of wobbles and makes an annoying sound like something is out of tolerance every few seconds at all RPM ranges.
----------------------------------------------------------------------------------------------------
B09VDNKL4G
Maybe a bad initial batch? They are very annoyingly noisey. I was so excited to get these after watching the youtube reviews at launch. Turns out I am in the same boat as other Amazon reviewers who are experiencing a lot of periodic resonance noise that is extremely annoying and what I would expect out of the cheapest noname brand type of fans or even a fan that is at the 